In [72]:
import pandas as pd
import numpy as np
import feather
import matplotlib as mpl
mpl.use('nbAgg')
import matplotlib.pyplot as plt
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import label_binarize
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
import math
from imblearn.combine import SMOTETomek
from sklearn.neural_network import MLPClassifier

%matplotlib inline

In [2]:
pingfInfoFilePath = './dv_personas_190411_previo_linear_regression'
dv_personas_190411_model = pd.read_feather(pingfInfoFilePath,columns=None, use_threads=True)
dv_personas_190411_model.head()

,sexo,tipo_acceso,tipo_erasmus,edad_fin_primera_titulacion,tto_pcto_dias_trabajados,dni,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),...,Titulacion_is_MÁSTER UNIVERSITARIO EN INVESTIGACIÓN EN HUMANIDADES,Titulacion_is_MÁSTER EN GESTIÓN DEL CONOCIMIENTO BIOMÉDICO E INVESTIGACIÓN CLÍNICA.,Titulacion_is_MÁSTER EN GESTIÓN Y DIRECCIÓN HOSTELERA,Titulacion_is_MÁSTER UNIVERSITARIO EN FORMACIÓN DEL PROFESORADO Y TIC,Titulacion_is_PCEO: DERECHO / ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS,Titulacion_is_PCEO: ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS / DERECHO,Titulacion_is_PCEO: ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS /CIENCIAS DEL TRABAJO,Titulacion_is_MÁSTER UNIVERSITARIO GESTIÓN CONOCIMIENTO BIOMÉDICO E INVESTIGACIÓN CLÍNICA,Titulacion_is_MÁSTER UNIVERSITARIO EN INVESTIGACIÓN EN CIENCIAS SOCIOSANITARIAS,Titulacion_is_MÁSTER UNIVERSITARIO EN PSICOPEDAGOGÍA
0,M,Selectividad,Ninguno,23,13.0,000b4ece399af8f6c90ad84acccc1aaea09d88ea83ac1c...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M,Selectividad,Ninguno,22,65.0,000e88e4e0244e5317dc105c2f77fdfde83fa66a60c813...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M,Selectividad,Ninguno,21,23.0,0013670b45b4207d01539bc1d057daa03bab0e2de19916...,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M,Formación Profesional,Ninguno,24,41.0,001d295ef6bbd65b0fb84c496ff81d6e9f2381e819289b...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M,Acceso a Segundo Ciclo,Ninguno,26,41.0,00230081e3257d2dd7d77d7722ce78b85545135946ed2e...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
categoricalCols = ['sexo','tipo_acceso','tipo_erasmus']

In [4]:
dv_personas_190411_model_one_hot = pd.get_dummies(data=dv_personas_190411_model,columns=categoricalCols)

In [5]:
dv_personas_190411_model_one_hot.head()

,edad_fin_primera_titulacion,tto_pcto_dias_trabajados,dni,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),Titulacion_is_GRADO EN MEDICINA,Titulacion_is_INGENIERO TÉCNICO INDUSTRIAL (ESPECIALIDAD EN ELECTRICIDAD),Titulacion_is_DIPLOMADO EN ENFERMERÍA,...,tipo_acceso_Estudios universitarios extranjeros parcialmente convalidados,tipo_acceso_Formación Profesional,tipo_acceso_Mayores de 25/40/45 años,tipo_acceso_Otros accesos a primer y segundo ciclo,tipo_acceso_Selectividad,tipo_acceso_Traslado de Expediente (Estudios Españoles),tipo_acceso_Título Universitario,tipo_erasmus_Estudios,tipo_erasmus_Ninguno,tipo_erasmus_Prácticas
0,23,13.0,000b4ece399af8f6c90ad84acccc1aaea09d88ea83ac1c...,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0
1,22,65.0,000e88e4e0244e5317dc105c2f77fdfde83fa66a60c813...,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
2,21,23.0,0013670b45b4207d01539bc1d057daa03bab0e2de19916...,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,24,41.0,001d295ef6bbd65b0fb84c496ff81d6e9f2381e819289b...,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,26,41.0,00230081e3257d2dd7d77d7722ce78b85545135946ed2e...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [6]:
dv_personas_190411_model_one_hot.drop(['dni'],axis=1,inplace = True)

#### Procedemos a dividir el tto_pcto_dias_trabajados en bloques de 33

In [7]:
tto_pcto_dias_trabajados_bucket_array = np.linspace(0,100,4)
tto_pcto_dias_trabajados_bucket_array

array([  0.        ,  33.33333333,  66.66666667, 100.        ])

In [8]:
dv_personas_190411_model_one_hot_33 = dv_personas_190411_model_one_hot.copy()
dv_personas_190411_model_one_hot_33['tto_pcto_dias_trabajados'] = pd.cut(dv_personas_190411_model_one_hot_33['tto_pcto_dias_trabajados'],tto_pcto_dias_trabajados_bucket_array,include_lowest=True)
dv_personas_190411_model_one_hot_33.head()

,edad_fin_primera_titulacion,tto_pcto_dias_trabajados,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),Titulacion_is_GRADO EN MEDICINA,Titulacion_is_INGENIERO TÉCNICO INDUSTRIAL (ESPECIALIDAD EN ELECTRICIDAD),Titulacion_is_DIPLOMADO EN ENFERMERÍA,Titulacion_is_DIPLOMADO EN TERAPIA OCUPACIONAL,...,tipo_acceso_Estudios universitarios extranjeros parcialmente convalidados,tipo_acceso_Formación Profesional,tipo_acceso_Mayores de 25/40/45 años,tipo_acceso_Otros accesos a primer y segundo ciclo,tipo_acceso_Selectividad,tipo_acceso_Traslado de Expediente (Estudios Españoles),tipo_acceso_Título Universitario,tipo_erasmus_Estudios,tipo_erasmus_Ninguno,tipo_erasmus_Prácticas
0,23,"(-0.001, 33.333]",0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
1,22,"(33.333, 66.667]",0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
2,21,"(-0.001, 33.333]",1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,24,"(33.333, 66.667]",0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,26,"(33.333, 66.667]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [9]:
dv_personas_190411_model_one_hot_33['tto_pcto_dias_trabajados'].astype('category')

0        (-0.001, 33.333]
1        (33.333, 66.667]
2        (-0.001, 33.333]
3        (33.333, 66.667]
4        (33.333, 66.667]
               ...       
14710    (-0.001, 33.333]
14711    (-0.001, 33.333]
14712    (-0.001, 33.333]
14713    (-0.001, 33.333]
14714    (-0.001, 33.333]
Name: tto_pcto_dias_trabajados, Length: 14715, dtype: category
Categories (3, interval[float64]): [(-0.001, 33.333] < (33.333, 66.667] < (66.667, 100.0]]

In [10]:
dv_personas_190411_model_one_hot_33['tto_pcto_dias_trabajados'] = dv_personas_190411_model_one_hot_33.tto_pcto_dias_trabajados.cat.codes
dv_personas_190411_model_one_hot_33.head()

,edad_fin_primera_titulacion,tto_pcto_dias_trabajados,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),Titulacion_is_GRADO EN MEDICINA,Titulacion_is_INGENIERO TÉCNICO INDUSTRIAL (ESPECIALIDAD EN ELECTRICIDAD),Titulacion_is_DIPLOMADO EN ENFERMERÍA,Titulacion_is_DIPLOMADO EN TERAPIA OCUPACIONAL,...,tipo_acceso_Estudios universitarios extranjeros parcialmente convalidados,tipo_acceso_Formación Profesional,tipo_acceso_Mayores de 25/40/45 años,tipo_acceso_Otros accesos a primer y segundo ciclo,tipo_acceso_Selectividad,tipo_acceso_Traslado de Expediente (Estudios Españoles),tipo_acceso_Título Universitario,tipo_erasmus_Estudios,tipo_erasmus_Ninguno,tipo_erasmus_Prácticas
0,23,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
1,22,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
2,21,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,24,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,26,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [11]:
dv_personas_190411_model_one_hot_33['tto_pcto_dias_trabajados'].value_counts()

0    9554
1    4121
2    1040
Name: tto_pcto_dias_trabajados, dtype: int64

Como se puede apreciar, hay un gran imbalance en las clases de los datos, la gran mayoría se encuentran en la clase 0 [0,33.33), dado a que las redes nuronales son sensibles a este tipo de casos habrá que aplicar alguna de las técnicas de resolución para este tipo de casos.

In [12]:
Features_1_2 = dv_personas_190411_model_one_hot_33.iloc[:,0:1]
Features_2_2 = dv_personas_190411_model_one_hot_33.iloc[:,2:]
Labels = dv_personas_190411_model_one_hot_33['tto_pcto_dias_trabajados']

In [13]:
Features = np.array(Features_1_2)
Features = np.concatenate([Features, np.array(Features_2_2)], axis = 1)
Features[:2,:]

array([[23,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  1,  0],
       [22,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
       

In [15]:
Features.shape

(14715, 204)

Procedemos a aplicar SMOTETomek que es una solución híbrida entre over-sampling y under-sampling

In [20]:
smt = SMOTETomek()
x_smt, y_smt = smt.fit_sample(Features, Labels)
print(x_smt.shape)
print(y_smt.shape)

(28576, 204)
(28576,)


In [24]:
y_smt_0 = y_smt[y_smt == 0]
y_smt_1 = y_smt[y_smt == 0]
y_smt_2 = y_smt[y_smt == 0]
print(y_smt_0.shape)
print(y_smt_1.shape)
print(y_smt_2.shape)

(9513,)
(9513,)
(9513,)


In [70]:
nr.seed(123)
inside = ms.KFold(n_splits=3, shuffle = True)
nr.seed(321)
outside = ms.KFold(n_splits=3, shuffle = True)

In [74]:
param_grid = {"alpha":[0.0000001,0.000001,0.00001], 
              "early_stopping":[True, False], 
              "beta_1":[0.95,0.90,0.80], 
              "beta_2":[0.999,0.9,0.8]}

## Define the Neural Network model
nn_clf = MLPClassifier(hidden_layer_sizes = (100,100),
                       max_iter=300)

## Perform the grid search over the parameters
nr.seed(3456)
nn_clf = ms.GridSearchCV(estimator = nn_clf, param_grid = param_grid, 
                      cv = inside, # Use the inside folds
                      scoring = 'roc_auc_ovo_weighted',
                      return_train_score = True)

nr.seed(6677)
nn_clf.fit(x_smt, y_smt)
print(nn_clf.best_estimator_.alpha)
print(nn_clf.best_estimator_.early_stopping)
print(nn_clf.best_estimator_.beta_1)
print(nn_clf.best_estimator_.beta_2)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, Conv

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


1e-07
False
0.8
0.999


In [ ]:
nr.seed(498)
cv_estimate = ms.cross_val_score(nn_clf, x_smt, y_smt, 
                                 cv = outside) # Use the outside folds

print('Mean performance metric = %4.3f' % np.mean(cv_estimate))
print('SDT of the metric       = %4.3f' % np.std(cv_estimate))
print('Outcomes by cv fold')
for i, x in enumerate(cv_estimate):
    print('Fold %2d    %4.3f' % (i+1, x))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, Conv

In [ ]:
nr.seed(9988)
indx = range(x_smt.shape[0])
indx = ms.train_test_split(indx, test_size = 3679)
x_train = x_smt[indx[0],:]
y_train = np.ravel(y_smt[indx[0]])
x_test = x_smt[indx[1],:]
y_test = np.ravel(y_smt[indx[1]])

In [ ]:
nr.seed(1115)
nn_mod = MLPClassifier(hidden_layer_sizes = (100,100), 
                       alpha = nn_clf.best_estimator_.alpha, 
                       early_stopping = nn_clf.best_estimator_.early_stopping, 
                       beta_1 = nn_clf.best_estimator_.beta_1, 
                       beta_2 = nn_clf.best_estimator_.beta_2,
                       max_iter = 300)
nn_mod.fit(x_train, y_train)